In [ ]:
class pair_trade:
    P_VALUE         = 0.05
    HURST           = 0.5                 # Checking Trend Persistance
    EXIT_PROFIT     = 0                   # Exit Point = 0 std
    TRADE_BOUND     = 1                   # Entry Point = 1 std
    STOP_LOSS       = 2                   # Stop Loss = 2 std
    TRADING_PERIOD  = 253

    def __init__ (self, price_df, trading_bound=TRADE_BOUND, exit_profit=EXIT_PROFIT, stop_loss=STOP_LOSS):
        
        self.asset_1_name  = price_df.columns[0]
        self.asset_2_name  = price_df.columns[1]

        self.price_df      = price_df
        self.trading_bound = trading_bound
        self.exit_profit   = exit_profit
        self.stop_loss     = stop_loss
        
        self.error = False
        try:
            beta = sm.OLS(self.price_df.iloc[:, 1], np.c_[np.ones((len(self.price_df))), self.price_df.iloc[:, 0]]).fit().params[1]
            self.spread = self.price_df.iloc[0, 1] - beta * self.price_df.iloc[:, 0]
            self.normalized_spread = (self.spread - self.spread.mean()) / self.spread.std()
            self.p_value = coint(self.price_df.iloc[:,0], self.price_df.iloc[:,1])[1]
            self.hurst = compute_Hc(self.spread)[0]
            self.generate_trading_signals()

        except Exception as e:
            print(e)
            print(f'Error!! Error!! Fucking Error!! with pairs {self.asset_1_name} & {self.asset_2_name}')
            self.error = True

        # beta = sm.OLS(self.price_df.iloc[:, 1], np.c_[np.ones((len(self.price_df))), self.price_df.iloc[:, 0]]).fit().params[1]
        # self.spread = self.price_df.iloc[0, 1] - beta * self.price_df.iloc[:, 0]
        # self.normalized_spread = (self.spread - self.spread.mean()) / self.spread.std()
        # self.p_value = coint(self.price_df.iloc[:,0], self.price_df.iloc[:,1])[1]
        # self.hurst = compute_Hc(self.spread)[0]
        # self.generate_trading_signals()

    def eligible (self, p_value = P_VALUE, hurst = HURST):
        if self.error:
            return False
        elif self.p_value <= p_value and self.hurst <= hurst:
            return True
        return False

    def level_crosses(self, series, level=2):
        change=[]
        for i, el in enumerate(series):
            if i != 0 and el > level and series[i-1] < level:
                change.append(1)
            elif i != 0 and el < level and series[i-1] > level:
                change.append(-1)
            else:
                change.append(0)
        return change

    def generate_trading_signals (self):

        upper_init = self.level_crosses(self.normalized_spread, level=  self.trading_bound)
        lower_init = self.level_crosses(self.normalized_spread, level= -self.trading_bound)
        upper_stop = self.level_crosses(self.normalized_spread, level=  self.stop_loss)
        lower_stop = self.level_crosses(self.normalized_spread, level= -self.stop_loss)
        mean       = self.level_crosses(self.normalized_spread, level=  self.exit_profit)

        open_position = False
        entry_level = 0

        self.stop_losses        = []
        self.open_positions     = []
        self.closed_positions   = []
        profits                 = []

        for i in range(0, len(self.normalized_spread)):
            if open_position:
                #stop loss
                if upper_stop[i] == 1 or lower_stop[i] == -1:
                    open_position = False
                    profits.append(-abs(self.spread[i] - entry_level))
                    self.stop_losses.append(i)
                #profit close
                elif mean[i] != 0:
                    open_position = False
                    profits.append(abs(self.spread[i] - entry_level))
                    self.closed_positions.append(i)
                else:
                    profits.append(0)

            else:
                #Entering The Position
                if upper_init[i] == -1 or lower_init[i] == 1:
                    open_position = True
                    entry_level = self.spread[i]
                    self.open_positions.append(i)
                profits.append(0)

        self.profit_series = pd.DataFrame(index=self.price_df.index, data=profits, columns=[f'{self.asset_1_name}_{self.asset_2_name}'])
        self.cum_profits = self.profit_series.cumsum()
        #self.sharp_ratio = 



In [ ]:
class pair_trade:
    P_VALUE         = 0.05
    HURST           = 0.5                 # Checking Trend Persistance
    EXIT_PROFIT     = 0                   # Exit Point = 0 std
    TRADE_BOUND     = 1                   # Entry Point = 1 std
    STOP_LOSS       = 2                   # Stop Loss = 2 std
    TRADING_PERIOD  = 253

    def __init__ (self, price_df, trading_bound=TRADE_BOUND, exit_profit=EXIT_PROFIT, stop_loss=STOP_LOSS):
        
        self.asset_1_name  = price_df.columns[0]
        self.asset_2_name  = price_df.columns[1]

        self.price_df      = price_df
        self.trading_bound = trading_bound
        self.exit_profit   = exit_profit
        self.stop_loss     = stop_loss
        
        self.error = False
        try:
            beta = sm.OLS(self.price_df.iloc[:, 1], np.c_[np.ones((len(self.price_df))), self.price_df.iloc[:, 0]]).fit().params[1]
            self.spread = self.price_df.iloc[0, 1] - beta * self.price_df.iloc[:, 0]
            self.normalized_spread = (self.spread - self.spread.mean()) / self.spread.std()
            self.p_value = coint(self.price_df.iloc[:,0], self.price_df.iloc[:,1])[1]
            self.hurst = compute_Hc(self.spread)[0]
            self.generate_trading_signals()

        except Exception as e:
            print(e)
            print(f'Error!! Error!! Fucking Error!! with pairs {self.asset_1_name} & {self.asset_2_name}')
            self.error = True

        # beta = sm.OLS(self.price_df.iloc[:, 1], np.c_[np.ones((len(self.price_df))), self.price_df.iloc[:, 0]]).fit().params[1]
        # self.spread = self.price_df.iloc[0, 1] - beta * self.price_df.iloc[:, 0]
        # self.normalized_spread = (self.spread - self.spread.mean()) / self.spread.std()
        # self.p_value = coint(self.price_df.iloc[:,0], self.price_df.iloc[:,1])[1]
        # self.hurst = compute_Hc(self.spread)[0]
        # self.generate_trading_signals()

    def eligible (self, p_value = P_VALUE, hurst = HURST):
        if self.error:
            return False
        elif self.p_value <= p_value and self.hurst <= hurst:
            return True
        return False

    def level_crosses(self, series, level=2):
        change=[]
        for i, el in enumerate(series):
            if i != 0 and el > level and series[i-1] < level:
                change.append(1)
            elif i != 0 and el < level and series[i-1] > level:
                change.append(-1)
            else:
                change.append(0)
        return change

    def generate_trading_signals (self):

        upper_init = self.level_crosses(self.normalized_spread, level=  self.trading_bound)
        lower_init = self.level_crosses(self.normalized_spread, level= -self.trading_bound)
        upper_stop = self.level_crosses(self.normalized_spread, level=  self.stop_loss)
        lower_stop = self.level_crosses(self.normalized_spread, level= -self.stop_loss)
        mean       = self.level_crosses(self.normalized_spread, level=  self.exit_profit)

        open_position = False
        entry_level = 0

        self.stop_losses        = []
        self.open_positions     = []
        self.closed_positions   = []
        profits                 = []

        for i in range(0, len(self.normalized_spread)):
            if open_position:
                #stop loss
                if upper_stop[i] == 1 or lower_stop[i] == -1:
                    open_position = False
                    profits.append(-abs(self.spread[i] - entry_level))
                    self.stop_losses.append(i)
                #profit close
                elif mean[i] != 0:
                    open_position = False
                    profits.append(abs(self.spread[i] - entry_level))
                    self.closed_positions.append(i)
                else:
                    profits.append(0)

            else:
                #Entering The Position
                if upper_init[i] == -1 or lower_init[i] == 1:
                    open_position = True
                    entry_level = self.spread[i]
                    self.open_positions.append(i)
                profits.append(0)

        self.profit_series = pd.DataFrame(index=self.price_df.index, data=profits, columns=[f'{self.asset_1_name}_{self.asset_2_name}'])
        self.cum_profits = self.profit_series.cumsum()
        #self.sharp_ratio = 

